# Training Pipeline

# Imports
## Pip Packages

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

if not os.path.exists('data'):
    new_directory_path = "..\\..\\"
    os.chdir(new_directory_path)

from src.datasets import FreeViewInMemory, seq2seq_jagged_collate_fn
from src.model import PathModel

# Dataset

## Load Dataset

In [2]:
# TODO Implement Strategy and Builder pattern
datasetv2 = FreeViewInMemory(sample_size= 13,log = True, start_index=2)


Data loaded in memory


# Training

In [3]:
model = PathModel(input_dim = 3,
                  output_dim = 3,
                  n_encoder = 2,
                  n_decoder = 2,
                  model_dim = 256,
                  total_dim = 256,
                  n_heads = 4,
                  ff_dim = 512,
                  max_pos_enc=15,
                  max_pos_dec=26)




In [4]:


dataloader = DataLoader(datasetv2, batch_size=128, shuffle=True, num_workers=0, collate_fn= seq2seq_jagged_collate_fn)
for batch in tqdm(dataloader):
    x,y = batch
    break
    

  0%|          | 0/274 [00:04<?, ?it/s]


In [5]:
model = torch.compile(model)
model(x,y)

W1106 20:29:08.959000 27400 site-packages\torch\nested\_internal\sdpa.py:327] [4/0] Memory efficient kernel not used because:
W1106 20:29:08.962000 27400 site-packages\torch\nested\_internal\sdpa.py:330] [4/0] Flash attention kernel not used because:
W1106 20:29:08.962000 27400 site-packages\torch\nested\_internal\sdpa.py:333] [4/0] Math attention kernel not used because:
W1106 20:29:08.967000 27400 site-packages\torch\nested\_internal\sdpa.py:280] [4/0] If inputs are nested tensors they must be contiguous after transposing.
W1106 20:29:08.968000 27400 site-packages\torch\nested\_internal\sdpa.py:335] [4/0] cuDNN attention kernel not used because:


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in function scaled_dot_product_attention>(*(NestedTensor(size=(128, 4, s1, 64), offsets=FakeTensor(..., size=(129,), dtype=torch.int64), grad_fn=<CloneBackward0 object at 0x000002CF04A9F970>, contiguous=True), NestedTensor(size=(128, 4, s1, 64), offsets=FakeTensor(..., size=(129,), dtype=torch.int64), grad_fn=<CloneBackward0 object at 0x000002CF04C223B0>, contiguous=True), NestedTensor(size=(128, 4, s1, 64), offsets=FakeTensor(..., size=(129,), dtype=torch.int64), grad_fn=<CloneBackward0 object at 0x000002CF04C21FC0>, contiguous=True)), **{'is_causal': False}): got RuntimeError('No viable backend for scaled_dot_product_attention was found.')

from user code:
   File "c:\Users\ulloa\OneDrive\Desktop\Practicas\projectes\Dataset\src\model.py", line 87, in forward
    x = self.norm1(x + self.__self_attention(x))
  File "c:\Users\ulloa\OneDrive\Desktop\Practicas\projectes\Dataset\src\model.py", line 76, 